<h1>Bridge to College</h1>

<h2>Work Items / Questions Completed:</h2>
•	Math in fresh year of college Compare L1, L2, L3 and L4 students in Bridge to College<br>
•	English in fresh year of college Compare L1, L2, L3 and L4 students in Bridge to College<br>
•	How many students that got a level 2 on their Math SBA in 11th grade took Bridge to college Math in 12th grade?<br>
•	How many students that got a level 2 on their English SBA in 11th grade took Bridge to college English in 12th grade?<br>
•	How did students do in Bridge? (Grade Distribution)<br>
•	What was the gender breakdown of the Bridge courses?<br>
•	What was the ethnic breakdown of the Bridge courses?<br>
•	Average grades for Bridge to College Math and Bridge to College ELA<br>
•	Remove any students with < 90 days in class<br>
•	Bridge to College that entered College <br>
•	Bridge Math in college (618 of 46.9%)(1263 or 47.82%)<br>
•	Bridge ELA in college (564 or 36.89%)(1379 or 39.52%)<br>
•	How many students graduated in 2016 that took B2C math and B2c ELA<br>
•	Were there discipline differences between the groups?<br>
•	What was the gender break-down of the non-bridge kids?<br>
•	What was the ethnic break-down of the non-bridge kids?<br>
•	Non-Bridge to College that entered College<br>
•	B or Better students that went to a CTC?<br>
•	Bridge B or Better students, who took math/ English 1 st quarter, and then who got what grade- same for the comparison group. <br>
•	Bridge course completion records ('W' for Grade)<br>
•	Community and Technical Colleges: (CTC)<br>
'Bellevue College', 'Big Bend Community College', 'Cascadia Community College', 'Centralia College', 'Clark College', 'Columbia Basin College', 'Edmonds Community College', 'Everett Community College', 'Grays Harbor College', 'Green River Community College', 'Highline Community College', 'Lower Columbia College', 'Olympic College', 'Peninsula College', 'Pierce College at Fort Steilacoom', 'Pierce College at Puyallup', 'Seattle Community College-Central Campus', 'Seattle Community College-North Campus', 'Seattle Community College-South Campus', 'Shoreline Community College', 'Skagit Valley College', 'South Puget Sound Community College', 'Spokane Community College', 'Spokane Falls Community College', 'Tacoma Community College', 'Walla Walla Community College', 'Wenatchee Valley College', 'Whatcom Community College', 'Yakima Valley Community College', 'Clover Park Technical College', 'Bellingham Technical College', 'Seattle Vocational 
Institute', 'Renton Technical College', 'Bates Technical College', 'Lake Washington Institute of Technology'<br>
•	Universities: (4 year colleges)<br>
'Eastern Washington University Semesterly (EWU)', 'The Evergreen State College (TESC)', 'University of Washington (UW)', 'Washington State University (WSU)', 'Western Washington University (WWU)', 'Eastern Washington University (EWU)', 'Central Washington University (CWU)'<br>

<h2>Work Items / Questions Left TODO</h2>
•	None<br>



<h2>Observations:</h2>
•	375 students took the Bridge Math course but not the Math SBA test.<br>
•	315 students took the Bridge English course but not the ELA SBA test.<br>



In [ ]:
print('Math: ' + str(math_df[(math_df.StateCourseCode == 2099) & (math_df.Subject != 'Smarter Balanced Math')]['ResearchID'].nunique()))
print('English: ' + str(english_df[(english_df.StateCourseCode == 1069) & (english_df.Subject != 'Smarter Balanced ELA')]['ResearchID'].nunique()))


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import time
import datetime
import csv
%matplotlib inline

source_path = 'D:\\PYTHON\\Bridge to College\\Data_Files\\' # Location of folder

source_discipline = 'BtC_Cohort 1_Discipline.csv' # Discipline file name
source_assess = 'BtC_Cohort 1_Assess.csv'
source_grades = 'BtC_Cohort 1_Grades.csv'
source_enrollment = 'BtC_Cohort 1_Enrollment.csv'
source_college = 'R3110_BERC_BTC_HEdetail.tab.D180311.T100410.ascii.T100410'

output_report = 'Output_Bridge_to_College_Report.csv' 
output_grade_report = 'Output_Bridge_Grades_Report.csv'
output_compare_report = 'Output_Compare_Report.csv'
output_college_grade_report = 'Output_College_Grades_Report.csv'

print('Execution Complete: ' + str(pd.datetime.now()))

In [ ]:
###################################
#####     Load Data Files     #####
###################################
start_time = time.time()
##### Assess file
print('Loading Assess File')
adf = pd.read_csv(source_path + source_assess)
adf['TestGrade'].replace(to_replace=['11'], value=11, inplace=True) 

##### Grades File
print('Loading Grades File')
gdf = pd.read_csv(source_path + source_grades)
gdf['StateCourseCode'].replace(to_replace=['02099'], value=2099, inplace=True)
gdf['StateCourseCode'].replace(to_replace=['01069'], value=1069, inplace=True)
gdf['LetterGrade'] = gdf['LetterGrade'].str.upper() 

##### Discipline File
print('Loading Discipline File')
ddf = pd.read_csv(source_path + source_discipline)

##### Enrollment File
print('Loading Enrollment Data')
edf = pd.read_csv(source_path + source_enrollment)
edf['GradeLevel'].replace(to_replace=['12'], value=12, inplace=True) 
edf['ExpectedGradYear'].replace(to_replace=['12'], value=12, inplace=True) 
#edf['NumDaysPresent'].replace(to_replace=[''], value=0, inplace=True) 
edf['NumDaysPresent'] = edf['NumDaysPresent'].apply(pd.to_numeric, errors='coerce')
#edf.fillna(0, inplace=True)

##### College File
print('Loading College file')
cdf = pd.read_csv(source_path + source_college, sep='\t', lineterminator='\r', encoding="ISO-8859-1")   

print('Duration of execution: ' + str(time.time() - start_time))
print('All files loaded: ' + str(datetime.datetime.now().strftime('%m/%d/%Y,  %H:%M.%S')))
print('Execution Complete: ' + str(pd.datetime.now()))

In [ ]:
#######################################################################################
##### Function to convert Letter grade to Number
def grade_letter_to_number(dataframe):    
    dataframe = dataframe[dataframe['LetterGrade'].isin(['P', 'W', 'NC', 'S', 'N']) == False] 
    dataframe['grade_point'] = dataframe['LetterGrade'].map(lambda x: 4 if ('A' in x)
                                                              else 3.7 if ('A-' in x)
                                                              else 3.3 if ('B+' in x)
                                                              else 3 if ('B' in x)
                                                              else 2.7 if ('B-' in x)
                                                              else 2.3 if ('C+' in x)
                                                              else 2 if ('C' in x)
                                                              else 1.7 if ('C-' in x)
                                                              else 1.3 if ('D+' in x)
                                                              else 1 if ('D' in x)
                                                              else 0.7 if ('D-' in x)
                                                              else 0 if ('F' in x)
                                                              else x)
    
    return dataframe
#######################################################################################
##### Function to Convert Number grade to Letter grade
def grade_number_to_letter(dataframe):    
    dataframe = dataframe[dataframe['NumAvgGrade'].isin(['P', 'W', 'NC', 'S', 'N']) == False] 
    dataframe['AvgGrade'] = dataframe['NumAvgGrade'].map(lambda x: 'A' if (4 == x)
                                                              else 'A-' if ((4 > x) & (3.66 <= x))
                                                              else 'B+' if ((3.66 > x) & (3.33 <= x))
                                                              else 'B' if ((3.33 > x) & (3 <= x))
                                                              else 'B-' if ((3 > x) & (2.66 <= x))
                                                              else 'C+' if ((2.66 > x) & (2.33 <= x))
                                                              else 'C' if ((2.33 > x) & (2 <= x))
                                                              else 'C-' if ((2 > x) & (1.66 <= x))
                                                              else 'D+' if ((1.66 > x) & (1.33 <= x))
                                                              else 'D' if ((1.33 > x) & (1 <= x))
                                                              else 'F' if (1 > x)
                                                              else x)
    
    return dataframe
#######################################################################################


In [ ]:

##########################################
#####  Create Math Data Frame     #####
##########################################
start_time = time.time()
#####################################
##### Identify Math Students ##### 
#####################################
print('Creating math Data Fame')
# Get the core data set, both Bridge and Non-Bridge
temp_edf = edf[((edf.NumDaysPresent >= 90) | (edf.WithdrawalCode.isin(['G0', 'GA', 'GB']))) & 
               (edf.SchoolYear == 2016) &
               (edf.ExpectedGradYear == 2016) & 
               (edf.GradeLevel == 12)][['ResearchID']]
print('Temp_edf Shape: ' + str(temp_edf.shape))

# Get the Bridge Students
temp_gdf = gdf[(gdf.SchoolYear == 2016) & 
               (gdf.StateCourseCode == 2099) &
               (gdf.GradeLevelWhenCourseTaken == 12)][['ResearchID']]
print('Temp_gdf Shape: ' + str(temp_gdf.shape))

math_df = temp_edf.append([temp_gdf])
print('Math Shape:  ' + str(math_df.shape))

math_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
print('Math Shape:  ' + str(math_df.shape))
##########################################
##### Adding Grades Data information #####
##########################################
print('Adding Grades File Data')
mgdf = gdf[(gdf.SchoolYear == 2016) & 
           (gdf.StateCourseCode == 2099) &
           (gdf.GradeLevelWhenCourseTaken == 12)][['ResearchID', 'Term', 'LetterGrade', 'StateCourseCode']]

## Save off the Grades not associated with Average Grade
temp_mgdf = mgdf[mgdf['LetterGrade'].isin(['P', 'W', 'NC', 'S', 'N']) != False] 
temp_mgdf.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
mgdf = grade_letter_to_number(mgdf)
for i in (mgdf['ResearchID']):
    (mgdf.loc[mgdf['ResearchID'] == i, 'NumAvgGrade']) = mgdf['grade_point'].loc[(mgdf['ResearchID'] == i)].mean(axis=0)
mgdf = grade_number_to_letter(mgdf)
mgdf.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
mgdf = mgdf[['ResearchID', 'AvgGrade', 'StateCourseCode']]
temp_mgdf = temp_mgdf.rename(columns = {'LetterGrade':'AvgGrade'})
temp_mgdf = temp_mgdf[temp_mgdf.columns.drop('Term')]
## Put Grades not associated with Average Grade back into Data Frame
mgdf = pd.merge(mgdf, temp_mgdf, how='outer')
mgdf.drop_duplicates(subset='ResearchID', keep='first', inplace=True)

math_df = pd.merge(math_df, mgdf, how = 'left', on='ResearchID')

print('Math Bridge: ' + str(math_df[math_df.StateCourseCode == 2099]['ResearchID'].nunique()))
print('Math Total:  ' + str(math_df.ResearchID.nunique()))
print('Math Shape:  ' + str(math_df.shape))
#########################################
#####     Add Enrollment File Data  #####
#########################################
print('Adding Enrollment File Data')
temp_df = edf[['ResearchID', 'Gender']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')

temp_df = edf[['ResearchID', 'FederalEthRaceRollupName']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')

temp_df = edf[edf.WithdrawalCode != ' '][['ResearchID', 'WithdrawalCode']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')

temp_df = edf[['ResearchID', 'NumDaysPresent']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')

temp_df = edf[['ResearchID', 'SchoolYear']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')

temp_df = edf[['ResearchID', 'ExpectedGradYear']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')

print('Math Bridge: ' + str(math_df[math_df.StateCourseCode == 2099]['ResearchID'].nunique()))
print('Math Total:  ' + str(math_df.ResearchID.nunique()))
print('Math Shape:  ' + str(math_df.shape))
###########################################
#####      Load Assess File Data      #####
###########################################
print('Adding Assess File Data')
temp_adf = adf[(adf.TestGrade == 11) & (adf.SchoolYear == 2015) & 
              (adf.Subject == 'Smarter Balanced Math')][['ResearchID', 'TestType', 'PerformanceLevel', 'Subject']]
                                                         
temp_df = temp_adf[['ResearchID', 'TestType']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')

temp_df = temp_adf[['ResearchID', 'PerformanceLevel']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')

temp_df = temp_adf[['ResearchID', 'Subject']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')

print('Math Bridge: ' + str(math_df[math_df.StateCourseCode == 2099]['ResearchID'].nunique()))
print('Math Total:  ' + str(math_df.ResearchID.nunique()))
print('Math Shape:  ' + str(math_df.shape))
##########################################
#####    Add Discipline File Data    #####
##########################################
print('Adding Discipline File  Data')
temp_df = ddf[(ddf.InterventionCode.isin(['SS', 'IS', 'LS', 'EX', 'EE'])) & 
              (ddf.SchoolYear == 2016)][['ResearchID', 'InterventionCode']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')

print('Math Bridge: ' + str(math_df[math_df.StateCourseCode == 2099]['ResearchID'].nunique()))
print('Math Total:  ' + str(math_df.ResearchID.nunique()))
print('Math Shape:  ' + str(math_df.shape))
###########################################
#####      Add College File Data      #####
###########################################
print('Adding College Information to the Data Frame')

temp_df = cdf[(cdf.TermTTL == 'Fall') & 
              (cdf.PreCollegeMathCourseFlag == 'Y')][['ResearchID', 'PreCollegeMathCourseFlag', 'GradeNumber']]
for i in (temp_df['ResearchID']):
    (temp_df.loc[temp_df['ResearchID'] == i, 'GradeNumber']) = temp_df['GradeNumber'].loc[(temp_df['ResearchID'] == i)].mean(axis=0)
temp_df = temp_df.rename(columns = {'PreCollegeMathCourseFlag':'FallPreCollege'})
temp_df = temp_df.rename(columns = {'GradeNumber':'FallPreCollegeGradeNumber'})
temp_df.drop_duplicates(subset='ResearchID', keep='first', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')
print('Add Fall PreCollege Shape: ' + str(math_df.shape))
print('ResearchID: ' + str(math_df.ResearchID.nunique()))
print('Math Shape:  ' + str(math_df.shape))

temp_df = cdf[(cdf.TermTTL == 'Winter') & 
              (cdf.PreCollegeMathCourseFlag == 'Y')][['ResearchID', 'PreCollegeMathCourseFlag', 'GradeNumber']]
for i in (temp_df['ResearchID']):
    (temp_df.loc[temp_df['ResearchID'] == i, 'GradeNumber']) = temp_df['GradeNumber'].loc[(temp_df['ResearchID'] == i)].mean(axis=0)
temp_df = temp_df.rename(columns = {'PreCollegeMathCourseFlag':'WinterPreCollege'})
temp_df = temp_df.rename(columns = {'GradeNumber':'WinterPreCollegeGradeNumber'})
temp_df.drop_duplicates(subset='ResearchID', keep='first', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')
print('Add Winter PreCollege Shape: ' + str(math_df.shape))
print('ResearchID: ' + str(math_df.ResearchID.nunique()))
print('Math Shape:  ' + str(math_df.shape))

temp_df = cdf[(cdf.TermTTL == 'Spring') & 
              (cdf.PreCollegeMathCourseFlag == 'Y')][['ResearchID', 'PreCollegeMathCourseFlag', 'GradeNumber']]
for i in (temp_df['ResearchID']):
    (temp_df.loc[temp_df['ResearchID'] == i, 'GradeNumber']) = temp_df['GradeNumber'].loc[(temp_df['ResearchID'] == i)].mean(axis=0)
temp_df = temp_df.rename(columns = {'PreCollegeMathCourseFlag':'SpringPreCollege'})
temp_df = temp_df.rename(columns = {'GradeNumber':'SpringPreCollegeGradeNumber'})
temp_df.drop_duplicates(subset='ResearchID', keep='first', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')
print('Add Spring PreCollege Shape: ' + str(math_df.shape))
print('ResearchID: ' + str(math_df.ResearchID.nunique()))
print('Math Shape:  ' + str(math_df.shape))

temp_df = cdf[(cdf.TermTTL == 'Fall') & 
              (cdf.CollegeLevelMathCourseEnrollment == 'Y')][['ResearchID', 'CollegeLevelMathCourseEnrollment', 'GradeNumber']]
for i in (temp_df['ResearchID']):
    (temp_df.loc[temp_df['ResearchID'] == i, 'GradeNumber']) = temp_df['GradeNumber'].loc[(temp_df['ResearchID'] == i)].mean(axis=0)
temp_df = temp_df.rename(columns = {'CollegeLevelMathCourseEnrollment':'FallCollege'})
temp_df = temp_df.rename(columns = {'GradeNumber':'FallCollegeGradeNumber'})
temp_df.drop_duplicates(subset='ResearchID', keep='first', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')
print('Add Fall College Shape: ' + str(math_df.shape))
print('ResearchID: ' + str(math_df.ResearchID.nunique()))
print('Math Shape:  ' + str(math_df.shape))

temp_df = cdf[(cdf.TermTTL == 'Winter') & 
              (cdf.CollegeLevelMathCourseEnrollment == 'Y')][['ResearchID', 'CollegeLevelMathCourseEnrollment', 'GradeNumber']]
for i in (temp_df['ResearchID']):
    (temp_df.loc[temp_df['ResearchID'] == i, 'GradeNumber']) = temp_df['GradeNumber'].loc[(temp_df['ResearchID'] == i)].mean(axis=0)
temp_df = temp_df.rename(columns = {'CollegeLevelMathCourseEnrollment':'WinterCollege'})
temp_df = temp_df.rename(columns = {'GradeNumber':'WinterCollegeGradeNumber'})
temp_df.drop_duplicates(subset='ResearchID', keep='first', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')
print('Add Winter College Shape: ' + str(math_df.shape))
print('ResearchID: ' + str(math_df.ResearchID.nunique()))
print('Math Shape:  ' + str(math_df.shape))

temp_df = cdf[(cdf.TermTTL == 'Spring') & 
              (cdf.CollegeLevelMathCourseEnrollment == 'Y')][['ResearchID', 'CollegeLevelMathCourseEnrollment', 'GradeNumber']]
for i in (temp_df['ResearchID']):
    (temp_df.loc[temp_df['ResearchID'] == i, 'GradeNumber']) = temp_df['GradeNumber'].loc[(temp_df['ResearchID'] == i)].mean(axis=0)
temp_df = temp_df.rename(columns = {'CollegeLevelMathCourseEnrollment':'SpringCollege'})
temp_df = temp_df.rename(columns = {'GradeNumber':'SpringCollegeGradeNumber'})
temp_df.drop_duplicates(subset='ResearchID', keep='first', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on='ResearchID')
print('Add Spring College Shape: ' + str(math_df.shape))
print('ResearchID: ' + str(math_df.ResearchID.nunique()))
print('Math Shape:  ' + str(math_df.shape))

temp_df = cdf[['ResearchID', 'EnrollmentOrganizationTTL']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
math_df = pd.merge(math_df, temp_df, how = 'left', on=['ResearchID'])
print('Add EnrollmentOrganizationTTL Shape:  ' + str(math_df.shape))

math_df.fillna('', inplace=True)
print('math_df ' + str(list(math_df.columns.values)))

print('Math Bridge: ' + str(math_df[math_df.StateCourseCode == 2099]['ResearchID'].nunique()))
print('Math Total:  ' + str(math_df.ResearchID.nunique()))
print('Final Math Shape:  ' + str(math_df.shape))
###########################################################################################################

print('Execution Duration: ' + str(time.time() - start_time))
print('Math Grades Data Frame Completed: ' + str(datetime.datetime.now().strftime('%m/%d/%Y,  %H:%M.%S')))


In [ ]:
#### ##########################################
#####  Create English Data Frame     #####
##########################################
start_time = time.time()
#####################################
##### Identify English Students ##### 
#####################################
print('Creating English Data Fame')
temp_edf = edf[((edf.NumDaysPresent >= 90) | (edf.WithdrawalCode.isin(['G0', 'GA', 'GB']))) & 
               (edf.SchoolYear == 2016) &
               (edf.ExpectedGradYear == 2016) & 
               (edf.GradeLevel == 12)][['ResearchID']]
print('Seniors Shape: ' + str(temp_edf.shape))

temp_gdf = gdf[(gdf.SchoolYear == 2016) & 
               (gdf.StateCourseCode == 1069) &
               (gdf.GradeLevelWhenCourseTaken == 12)][['ResearchID']]
print('Bridge shape: ' + str(temp_gdf.shape))
english_df = temp_edf.append([temp_gdf])
print('english_df shape: ' + str(english_df.shape))
english_df.drop_duplicates(subset=None, keep='last', inplace=True)
print('english_df shape: ' + str(english_df.shape))
##########################################
##### Adding Grades Data information #####
##########################################
print('Adding Grades File Data')
egdf = gdf[(gdf.SchoolYear == 2016) & 
           (gdf.StateCourseCode == 1069) &
           (gdf.GradeLevelWhenCourseTaken == 12)][['ResearchID', 'Term', 'LetterGrade', 'StateCourseCode']]

## Save off the Grades not associated with Average Grade
temp_egdf = egdf[egdf['LetterGrade'].isin(['P', 'W', 'NC', 'S', 'N']) != False] 
temp_egdf.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
egdf = grade_letter_to_number(egdf)
for i in (egdf['ResearchID']):
    (egdf.loc[egdf['ResearchID'] == i, 'NumAvgGrade']) = egdf['grade_point'].loc[(egdf['ResearchID'] == i)].mean(axis=0)
egdf = grade_number_to_letter(egdf)
egdf.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
egdf = egdf[['ResearchID', 'AvgGrade', 'StateCourseCode']]
temp_egdf = temp_egdf.rename(columns = {'LetterGrade':'AvgGrade'})
temp_egdf = temp_egdf[temp_egdf.columns.drop('Term')]
## Put Grades not associated with Average Grade back into Data Frame
egdf = pd.merge(egdf, temp_egdf, how='outer')
egdf.drop_duplicates(subset='ResearchID', keep='first', inplace=True)

english_df = pd.merge(english_df, egdf, how = 'left', on='ResearchID')

print('English Shape:  ' + str(english_df.shape))
print('English Bridge: ' + str(english_df[english_df.StateCourseCode == 1069]['ResearchID'].nunique()))
print('English Total:  ' + str(english_df.ResearchID.nunique()))
#########################################
#####     Add Enrollment File Data  #####
#########################################
print('Adding Enrollment File Data')
temp_df = edf[['ResearchID', 'Gender']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')

temp_df = edf[['ResearchID', 'FederalEthRaceRollupName']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')

temp_df = edf[edf.WithdrawalCode != ' '][['ResearchID', 'WithdrawalCode']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')

temp_df = edf[['ResearchID', 'NumDaysPresent']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')

temp_df = edf[['ResearchID', 'SchoolYear']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')

temp_df = edf[['ResearchID', 'ExpectedGradYear']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')

print('English Shape:  ' + str(english_df.shape))
print('English Bridge: ' + str(english_df[english_df.StateCourseCode == 1069]['ResearchID'].nunique()))
print('English Total:  ' + str(english_df.ResearchID.nunique()))
###########################################
#####      Load Assess File Data      #####
###########################################
print('Adding Assess File Data')
temp_adf = adf[(adf.TestGrade == 11) & (adf.SchoolYear == 2015) & 
              (adf.Subject == 'Smarter Balanced ELA')][['ResearchID', 'TestType', 'PerformanceLevel', 'Subject']]
                                                         
temp_df = temp_adf[['ResearchID', 'TestType']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')

temp_df = temp_adf[['ResearchID', 'PerformanceLevel']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')

temp_df = temp_adf[['ResearchID', 'Subject']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')

print('English Shape:  ' + str(english_df.shape))
print('English Bridge: ' + str(english_df[english_df.StateCourseCode == 1069]['ResearchID'].nunique()))
print('English Total:  ' + str(english_df.ResearchID.nunique()))
##########################################
#####    Add Discipline File Data    #####
##########################################
print('Adding Discipline File  Data')
temp_df = ddf[(ddf.InterventionCode.isin(['SS', 'IS', 'LS', 'EX', 'EE'])) & 
              (ddf.SchoolYear == 2016)][['ResearchID', 'InterventionCode']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')

print('English Shape:  ' + str(english_df.shape))
print('English Bridge: ' + str(english_df[english_df.StateCourseCode == 1069]['ResearchID'].nunique()))
print('English Total:  ' + str(english_df.ResearchID.nunique()))
###########################################
#####      Add College File Data      #####
###########################################
print('Adding College Information to the Data Frame')

temp_df = cdf[(cdf.TermTTL == 'Fall') & 
              (cdf.PreCollegeEnglishCourseFlag == 'Y')][['ResearchID', 'PreCollegeEnglishCourseFlag', 'GradeNumber']]
for i in (temp_df['ResearchID']):
    (temp_df.loc[temp_df['ResearchID'] == i, 'GradeNumber']) = temp_df['GradeNumber'].loc[(temp_df['ResearchID'] == i)].mean(axis=0)
temp_df = temp_df.rename(columns = {'PreCollegeEnglishCourseFlag':'FallPreCollege'})
temp_df = temp_df.rename(columns = {'GradeNumber':'FallPreCollegeGradeNumber'})
temp_df.drop_duplicates(subset='ResearchID', keep='first', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')
print('Add Fall PreCollege Shape: ' + str(english_df.shape))
print('ResearchID: ' + str(english_df.ResearchID.nunique()))
print('English Shape:  ' + str(english_df.shape))

temp_df = cdf[(cdf.TermTTL == 'Winter') & 
              (cdf.PreCollegeEnglishCourseFlag == 'Y')][['ResearchID', 'PreCollegeEnglishCourseFlag', 'GradeNumber']]
for i in (temp_df['ResearchID']):
    (temp_df.loc[temp_df['ResearchID'] == i, 'GradeNumber']) = temp_df['GradeNumber'].loc[(temp_df['ResearchID'] == i)].mean(axis=0)
temp_df = temp_df.rename(columns = {'PreCollegeEnglishCourseFlag':'WinterPreCollege'})
temp_df = temp_df.rename(columns = {'GradeNumber':'WinterPreCollegeGradeNumber'})
temp_df.drop_duplicates(subset='ResearchID', keep='first', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')
print('Add Winter PreCollege Shape: ' + str(english_df.shape))
print('ResearchID: ' + str(english_df.ResearchID.nunique()))
print('English Shape:  ' + str(english_df.shape))

temp_df = cdf[(cdf.TermTTL == 'Spring') & 
              (cdf.PreCollegeEnglishCourseFlag == 'Y')][['ResearchID', 'PreCollegeEnglishCourseFlag', 'GradeNumber']]
for i in (temp_df['ResearchID']):
    (temp_df.loc[temp_df['ResearchID'] == i, 'GradeNumber']) = temp_df['GradeNumber'].loc[(temp_df['ResearchID'] == i)].mean(axis=0)
temp_df = temp_df.rename(columns = {'PreCollegeEnglishCourseFlag':'SpringPreCollege'})
temp_df = temp_df.rename(columns = {'GradeNumber':'SpringPreCollegeGradeNumber'})
temp_df.drop_duplicates(subset='ResearchID', keep='first', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')
print('Add Spring PreCollege Shape: ' + str(english_df.shape))
print('ResearchID: ' + str(english_df.ResearchID.nunique()))
print('English Shape:  ' + str(english_df.shape))

temp_df = cdf[(cdf.TermTTL == 'Fall') & 
              (cdf.CollegeLevelEnglishCourseEnrollm == 'Y')][['ResearchID', 'CollegeLevelEnglishCourseEnrollm', 'GradeNumber']]
for i in (temp_df['ResearchID']):
    (temp_df.loc[temp_df['ResearchID'] == i, 'GradeNumber']) = temp_df['GradeNumber'].loc[(temp_df['ResearchID'] == i)].mean(axis=0)
temp_df = temp_df.rename(columns = {'CollegeLevelEnglishCourseEnrollm':'FallCollege'})
temp_df = temp_df.rename(columns = {'GradeNumber':'FallCollegeGradeNumber'})
temp_df.drop_duplicates(subset='ResearchID', keep='first', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')
print('Add Fall College Shape: ' + str(english_df.shape))
print('ResearchID: ' + str(english_df.ResearchID.nunique()))
print('English Shape:  ' + str(english_df.shape))

temp_df = cdf[(cdf.TermTTL == 'Winter') & 
              (cdf.CollegeLevelEnglishCourseEnrollm == 'Y')][['ResearchID', 'CollegeLevelEnglishCourseEnrollm', 'GradeNumber']]
for i in (temp_df['ResearchID']):
    (temp_df.loc[temp_df['ResearchID'] == i, 'GradeNumber']) = temp_df['GradeNumber'].loc[(temp_df['ResearchID'] == i)].mean(axis=0)
temp_df = temp_df.rename(columns = {'CollegeLevelEnglishCourseEnrollm':'WinterCollege'})
temp_df = temp_df.rename(columns = {'GradeNumber':'WinterCollegeGradeNumber'})
temp_df.drop_duplicates(subset='ResearchID', keep='first', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')
print('Add Winter College Shape: ' + str(english_df.shape))
print('ResearchID: ' + str(english_df.ResearchID.nunique()))
print('English Shape:  ' + str(english_df.shape))

temp_df = cdf[(cdf.TermTTL == 'Spring') & 
              (cdf.CollegeLevelEnglishCourseEnrollm == 'Y')][['ResearchID', 'CollegeLevelEnglishCourseEnrollm', 'GradeNumber']]
for i in (temp_df['ResearchID']):
    (temp_df.loc[temp_df['ResearchID'] == i, 'GradeNumber']) = temp_df['GradeNumber'].loc[(temp_df['ResearchID'] == i)].mean(axis=0)
temp_df = temp_df.rename(columns = {'CollegeLevelEnglishCourseEnrollm':'SpringCollege'})
temp_df = temp_df.rename(columns = {'GradeNumber':'SpringCollegeGradeNumber'})
temp_df.drop_duplicates(subset='ResearchID', keep='first', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on='ResearchID')
print('Add Spring College Shape: ' + str(english_df.shape))
print('ResearchID: ' + str(english_df.ResearchID.nunique()))
print('English Shape:  ' + str(english_df.shape))

temp_df = cdf[['ResearchID', 'EnrollmentOrganizationTTL']]
temp_df.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
english_df = pd.merge(english_df, temp_df, how = 'left', on=['ResearchID'])
print('Add EnrollmentOrganizationTTL Shape:  ' + str(english_df.shape))


english_df.fillna('', inplace=True)

print('Final English Shape:  ' + str(english_df.shape))
print('English Bridge: ' + str(english_df[english_df.StateCourseCode == 1069]['ResearchID'].nunique()))
print('English Total:  ' + str(english_df.ResearchID.nunique()))
#################################################################################################################################

print('Execution Duration: ' + str(time.time() - start_time))
print('english Grades Data Frame Completed: ' + str(datetime.datetime.now().strftime('%m/%d/%Y,  %H:%M.%S')))


In [ ]:
###############################################
#####     Functions to write row data     #####
###############################################
def write_row (group, category_value, num_students, num_grad_hs, percent_grad_hs, num_college, percent_college, 
               percent_ctc_college, percent_university, percent_precollege_fall, percent_college_fall, percent_precollege_spring, 
               percent_college_spring, num_discipline, percent_discipline):
    writer.writerow({'Student_Group': group, 
                     'Category Value':category_value,
                     'Total Students': num_students, 
                     'Graduated HS':num_grad_hs,
                     '% Graduated HS':percent_grad_hs,
                     'Entered College': num_college,
                     '% Entered College': percent_college, 
                     '% CTC College': percent_ctc_college,
                     '% University': percent_university,
                     'PreCollege Level % Fall': percent_precollege_fall, 
                     'College Level % Fall': percent_college_fall, 
                     'PreCollege Level % Spring': percent_precollege_spring,
                     'College Level % Spring': percent_college_spring,
                     'Discipline Issues':num_discipline,
                     '% Discipline':percent_discipline})

#######################################################
#####            Write a simple row Row           #####
#######################################################
def write_line (row_name):
    write_row(row_name, '', '', '', '', '', '', '', '', '', '', '', '', '', '')

def write_reference_category_row(row_name, category_value):
    
    total = temp_df.ResearchID.nunique()
    num_grad_hs = temp_df[(temp_df.WithdrawalCode.isin(['G0', 'GA', 'GB']))]['ResearchID'].nunique()
    percent_grad_hs = str('%4.2f' %(100 * (num_grad_hs / total)))
    num_college = temp_df[(temp_df.EnrollmentOrganizationTTL != '')]['ResearchID'].nunique()
    percent_college = str('%4.2f' %(100 * (num_college / total))) 
    num_ctc_college = temp_df[(temp_df.EnrollmentOrganizationTTL.isin(['CLARK', 'SPOKANE FALLS', 'EVERETT', 'SPOKANE COMMUNITY', 
                                                                   'OLYMPIC', 'SHORELINE', 'SKAGIT VALLEY', 'TACOMA', 
                                                                   'EDMONDS', 'YAKIMA VALLEY', 'HIGHLINE', 'WALLA WALLA', 
                                                                   'LOWER COLUMBIA', 'PIERCE FORT STEILACOOM', 'BIG BEND', 
                                                                   'BELLEVUE', 'GREEN RIVER', 'PENINSULA', 'PIERCE PUYALLUP', 
                                                                   'CENTRALIA', 'SEATTLE CENTRAL', 'SEATTLE NORTH', 
                                                                   'GRAYS HARBOR', 'SOUTH PUGET SOUND', 'CASCADIA', 'WHATCOM', 
                                                                   'CLOVER PARK', 'SEATTLE SOUTH', 'BATES', 'COLUMBIA BASIN', 
                                                                   'LAKE WASHINGTON', 'BELLINGHAM', 'RENTON', 
                                                                   'WENATCHEE VALLEY', 'SEATTLE VOC INSTITUTE', 
                                                                   'Eastern Washington University Semesterly (EWU)']))]['ResearchID'].nunique()
    percent_ctc_college = str('%4.2f' %(100 * (num_ctc_college / num_college))) 

    num_university = temp_df[(temp_df.EnrollmentOrganizationTTL.isin(['The Evergreen State College (TESC)', 
                                                                      'University of Washington (UW)', 
                                                                      'Washington State University (WSU)', 
                                                                      'Western Washington University (WWU)', 
                                                                      'Eastern Washington University (EWU)', 
                                                                      'Central Washington University (CWU)']))]['ResearchID'].nunique()
    percent_university = str('%4.2f' %(100 * (num_university / num_college))) 

    percent_precollege_fall = str('%4.2f' %(100 * ((temp_df[((temp_df['FallPreCollege'] == 'Y'))]
                                                    ['ResearchID'].nunique()) / total)))
    percent_college_fall = str('%4.2f' %(100 * ((temp_df[((temp_df['FallCollege'] == 'Y'))]
                                                    ['ResearchID'].nunique()) / total)))
    percent_precollege_spring = str('%4.2f' %(100 * ((temp_df[((temp_df['SpringPreCollege'] == 'Y'))]
                                                    ['ResearchID'].nunique()) / total)))
    percent_college_spring = str('%4.2f' %(100 * ((temp_df[((temp_df['SpringCollege'] == 'Y'))]
                                                    ['ResearchID'].nunique()) / total)))    
    num_discipline = temp_df[(temp_df.InterventionCode.isin(['SS', 'IS', 'LS', 'EX', 'EE']))]['ResearchID'].nunique()
    percent_discipline = str('%4.2f' %(100 * (num_discipline / total)))
    write_row(row_name, category_value, total, num_grad_hs, percent_grad_hs, num_college, percent_college, 
              percent_ctc_college, percent_university, percent_precollege_fall, percent_college_fall, 
              percent_precollege_spring, percent_college_spring, num_discipline, percent_discipline)

    
######################################################    
def write_category_row(row_name, category, category_value):
    
    total = temp_df[temp_df[category] == category_value]['ResearchID'].nunique()
    num_grad_hs = temp_df[(temp_df[category] == category_value) & 
                          (temp_df.WithdrawalCode.isin(['G0', 'GA', 'GB']))]['ResearchID'].nunique()
    percent_grad_hs = str('%4.2f' %(100 * (num_grad_hs / total)))
    num_college = temp_df[(temp_df[category] == category_value) & 
                          (temp_df.EnrollmentOrganizationTTL != '')]['ResearchID'].nunique()
    percent_college = str('%4.2f' %(100 * (num_college / total))) 
    num_ctc_college = temp_df[(temp_df[category] == category_value) & 
                          (temp_df.EnrollmentOrganizationTTL.isin(['CLARK', 'SPOKANE FALLS', 'EVERETT', 'SPOKANE COMMUNITY', 
                                                                   'OLYMPIC', 'SHORELINE', 'SKAGIT VALLEY', 'TACOMA', 
                                                                   'EDMONDS', 'YAKIMA VALLEY', 'HIGHLINE', 'WALLA WALLA', 
                                                                   'LOWER COLUMBIA', 'PIERCE FORT STEILACOOM', 'BIG BEND', 
                                                                   'BELLEVUE', 'GREEN RIVER', 'PENINSULA', 'PIERCE PUYALLUP', 
                                                                   'CENTRALIA', 'SEATTLE CENTRAL', 'SEATTLE NORTH', 
                                                                   'GRAYS HARBOR', 'SOUTH PUGET SOUND', 'CASCADIA', 'WHATCOM', 
                                                                   'CLOVER PARK', 'SEATTLE SOUTH', 'BATES', 'COLUMBIA BASIN', 
                                                                   'LAKE WASHINGTON', 'BELLINGHAM', 'RENTON', 
                                                                   'WENATCHEE VALLEY', 'SEATTLE VOC INSTITUTE', 
                                                                   'Eastern Washington University Semesterly (EWU)']))]['ResearchID'].nunique()
    
    num_university = temp_df[(temp_df[category] == category_value) & 
                             (temp_df.EnrollmentOrganizationTTL.isin(['The Evergreen State College (TESC)', 
                                                                      'University of Washington (UW)', 
                                                                      'Washington State University (WSU)', 
                                                                      'Western Washington University (WWU)', 
                                                                      'Eastern Washington University (EWU)', 
                                                                      'Central Washington University (CWU)']))]['ResearchID'].nunique()
    percent_university = str('%4.2f' %(100 * (num_university / num_college))) 

    percent_ctc_college = str('%4.2f' %(100 * (num_ctc_college / num_college))) 
    percent_precollege_fall = str('%4.2f' %(100 * ((temp_df[(temp_df[category] == category_value) & 
                                                               (temp_df['FallPreCollege'] == 'Y')]
                                                                ['ResearchID'].nunique()) / total)))
    percent_college_fall = str('%4.2f' %(100 * ((temp_df[(temp_df[category] == category_value) & 
                                                               (temp_df['FallCollege'] == 'Y')]
                                                                ['ResearchID'].nunique()) / total)))
    percent_precollege_spring = str('%4.2f' %(100 * ((temp_df[(temp_df[category] == category_value) & 
                                                               (temp_df['SpringPreCollege'] == 'Y')]
                                                                ['ResearchID'].nunique()) / total)))
    percent_college_spring = str('%4.2f' %(100 * ((temp_df[(temp_df[category] == category_value) & 
                                                               (temp_df['SpringCollege'] == 'Y')]
                                                                ['ResearchID'].nunique()) / total)))
    num_discipline = temp_df[(temp_df[category] == category_value) &  
                             (temp_df.InterventionCode.isin(['SS', 'IS', 'LS', 'EX', 'EE']))]['ResearchID'].nunique()
    percent_discipline = str('%4.2f' %(100 * (num_discipline / total)))
    write_row(row_name, category_value, total, num_grad_hs, percent_grad_hs, num_college, percent_college, 
              percent_ctc_college, percent_university, percent_precollege_fall, percent_college_fall, 
              percent_precollege_spring, percent_college_spring, num_discipline, percent_discipline)

#####################################################
#####     Set up and Create the College Report  #####
#####################################################

with open(source_path + output_report, "w", newline='') as csvfile:
    fieldnames = ['Student_Group', 'Category Value', 'Total Students', 'Graduated HS', '% Graduated HS', 
                  'Entered College', '% Entered College', '% CTC College', '% University', 'PreCollege Level % Fall', 
                  'College Level % Fall', 'PreCollege Level % Spring', 'College Level % Spring', 'Discipline Issues', 
                  '% Discipline']
    
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
####################     English Data     ####################
    write_line ('English')
    # English All
    temp_df = english_df[(english_df.StateCourseCode == 1069) | ((english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & 
                         (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0))]
    write_reference_category_row('All English', 'All Students')

    #  English Non-Bridge
    english_df['NumDaysPresent'] = english_df['NumDaysPresent'].apply(pd.to_numeric, errors='coerce')
    temp_df = english_df[(english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & 
                         (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0)]
    write_reference_category_row('English Non-Bridge', 'All Non-Bridge')

    #  English Bridge SBA
    temp_df = english_df[(english_df.StateCourseCode == 1069)]
    write_reference_category_row('English Bridge', 'All Bridge')
    write_category_row('English Bridge SBA Level', 'PerformanceLevel', 'L1')
    write_category_row('English Bridge SBA Level', 'PerformanceLevel', 'L2')
    write_category_row('English Bridge SBA Level', 'PerformanceLevel', 'L3')
    write_category_row('English Bridge SBA Level', 'PerformanceLevel', 'L4')
    write_category_row('English Bridge SBA Level', 'PerformanceLevel', 'BA')
    write_category_row('English Bridge SBA Level', 'PerformanceLevel', '')
    
    #  English Bridge B or Better SBA
    temp_df = english_df[(english_df.StateCourseCode == 1069) & (english_df.AvgGrade.isin(['A', 'A-', 'B+', 'B']))]
    write_reference_category_row('English Bridge B or Better SBA Level', 'All Levels')
    write_category_row('English Bridge B or Better SBA Level', 'PerformanceLevel', 'L1')
    write_category_row('English Bridge B or Better SBA Level', 'PerformanceLevel', 'L2')
    write_category_row('English Bridge B or Better SBA Level', 'PerformanceLevel', 'L3')
    write_category_row('English Bridge B or Better SBA Level', 'PerformanceLevel', 'L4')
    write_category_row('English Bridge B or Better SBA Level', 'PerformanceLevel', 'BA')
    write_category_row('English Bridge B or Better SBA Level', 'PerformanceLevel', '')

    #  English Bridge Gender
    temp_df = english_df[(english_df.StateCourseCode == 1069)]
    write_category_row('English Bridge Gender', 'Gender', 'F')
    write_category_row('English Bridge Gender', 'Gender', 'M')
    
    #  English Non-Bridge Gender
    english_df['NumDaysPresent'] = english_df['NumDaysPresent'].apply(pd.to_numeric, errors='coerce')
    temp_df = english_df[(english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & 
                         (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0)]
    write_category_row('English Non-Bridge Gender', 'Gender', 'F')
    write_category_row('English Non-Bridge Gender', 'Gender', 'M')

    # English Bridge FederalEthRaceRollupName
    temp_df = english_df[(english_df.StateCourseCode == 1069)]
    write_category_row('English Bridge Ethnic Race', 'FederalEthRaceRollupName', 'White')
    write_category_row('English Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Hispanic/Latino of any race(s)')
    write_category_row('English Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Two or More Races')
    write_category_row('English Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Black/African American')
    write_category_row('English Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Asian')
    write_category_row('English Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Native Hawaiian/Other Pacific Islander')
    write_category_row('English Bridge Ethnic Race', 'FederalEthRaceRollupName', 'American Indian/Alaskan Native')
    
    # English Non-Bridge FederalEthRaceRollupName
    english_df['NumDaysPresent'] = english_df['NumDaysPresent'].apply(pd.to_numeric, errors='coerce')
    temp_df = english_df[(english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & 
                         (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0)]
    write_category_row('English Non-Bridge Ethnic Race', 'FederalEthRaceRollupName', 'White')
    write_category_row('English Non-Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Hispanic/Latino of any race(s)')
    write_category_row('English Non-Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Two or More Races')
    write_category_row('English Non-Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Black/African American')
    write_category_row('English Non-Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Asian')
    write_category_row('English Non-Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Native Hawaiian/Other Pacific Islander')
    write_category_row('English Non-Bridge Ethnic Race', 'FederalEthRaceRollupName', 'American Indian/Alaskan Native')

    write_line ('')
    write_line ('')

 ####################     Math Data     ####################
    write_line ('Math')
    #  Math All
    temp_df = math_df[(math_df.StateCourseCode == 2099) | ((math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & 
                         (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0))]
    write_reference_category_row('All Math', 'All Students')
    
    # Math Non-Bridge FederalEthRaceRollupName
    math_df['NumDaysPresent'] = math_df['NumDaysPresent'].apply(pd.to_numeric, errors='coerce')
    temp_df = math_df[(math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & 
                         (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0)]
    write_reference_category_row('Math Non-Bridge', 'All Non-Bridge')

    #  Math Bridge
    temp_df = math_df[(math_df.StateCourseCode == 2099)]
    write_reference_category_row('Math Bridge', 'All Bridge')
    
    #  Math Bridge SBA
    temp_df = math_df[(math_df.StateCourseCode == 2099)]
    write_category_row('Math Bridge SBA Level', 'PerformanceLevel', 'L1')
    write_category_row('Math Bridge SBA Level', 'PerformanceLevel', 'L2')
    write_category_row('Math Bridge SBA Level', 'PerformanceLevel', 'L3')
    write_category_row('Math Bridge SBA Level', 'PerformanceLevel', 'L4')
#    write_category_row('Math Bridge SBA Level', 'PerformanceLevel', 'BA')
    write_category_row('Math Bridge SBA Level', 'PerformanceLevel', '')
    
    #  Math Bridge B or Better
    temp_df = math_df[(math_df.StateCourseCode == 2099) & (math_df.AvgGrade.isin(['A', 'A-', 'B+', 'B']))]
    write_reference_category_row('Math Bridge B or Better SBA Level', 'All Levels')
    write_category_row('Math Bridge B or Better SBA Level', 'PerformanceLevel', 'L1')
    write_category_row('Math Bridge B or Better SBA Level', 'PerformanceLevel', 'L2')
    write_category_row('Math Bridge B or Better SBA Level', 'PerformanceLevel', 'L3')
    write_category_row('Math Bridge B or Better SBA Level', 'PerformanceLevel', 'L4')
#    write_category_row('Math Bridge B or Better SBA Level', 'PerformanceLevel', 'BA')
    write_category_row('Math Bridge B or Better SBA Level', 'PerformanceLevel', '')


    #  Math Bridge Gender
    temp_df = math_df[(math_df.StateCourseCode == 2099)]
    write_category_row('Math Bridge Gender', 'Gender', 'F')
    write_category_row('Math Bridge Gender', 'Gender', 'M')

    #  Math Non Bridge Gender
    math_df['NumDaysPresent'] = math_df['NumDaysPresent'].apply(pd.to_numeric, errors='coerce')
    temp_df = math_df[(math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & 
                         (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0)]
    write_category_row('Math Non-Bridge Gender', 'Gender', 'F')
    write_category_row('Math Non-Bridge Gender', 'Gender', 'M')

    #  Math Bridge FederalEthRaceRollupName
    temp_df = math_df[(math_df.StateCourseCode == 2099)]
    write_category_row('Math Bridge Ethnic Race', 'FederalEthRaceRollupName', 'White')
    write_category_row('Math Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Hispanic/Latino of any race(s)')
    write_category_row('Math Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Two or More Races')
    write_category_row('Math Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Black/African American')
    write_category_row('Math Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Asian')
    write_category_row('Math Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Native Hawaiian/Other Pacific Islander')
    write_category_row('Math Bridge Ethnic Race', 'FederalEthRaceRollupName', 'American Indian/Alaskan Native')
    
    # Math Non-Bridge FederalEthRaceRollupName
    math_df['NumDaysPresent'] = math_df['NumDaysPresent'].apply(pd.to_numeric, errors='coerce')
    temp_df = math_df[(math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & 
                         (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0)]
    write_category_row('Math Non-Bridge Ethnic Race', 'FederalEthRaceRollupName', 'White')
    write_category_row('Math Non-Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Hispanic/Latino of any race(s)')
    write_category_row('Math Non-Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Two or More Races')
    write_category_row('Math Non-Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Black/African American')
    write_category_row('Math Non-Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Asian')
    write_category_row('Math Non-Bridge Ethnic Race', 'FederalEthRaceRollupName', 'Native Hawaiian/Other Pacific Islander')
    write_category_row('Math Non-Bridge Ethnic Race', 'FederalEthRaceRollupName', 'American Indian/Alaskan Native')


    csvfile.close()
print('Execution Completed: ' + str(datetime.datetime.now().strftime('%m/%d/%Y,  %H:%M.%S')))
    


In [ ]:
####################################################
#####     Set up and Create the Grades Report  #####
####################################################

#############################################
#####     Functions to write row data   #####
#############################################
def write_grade_row (group, level, num_students, A, Aminus, Bplus, B, Bminus, Cplus, C, Cminus, Dplus, D, F, NC, N, P, S, W): 
    writer.writerow({'Student_Group': group, 
                     'Category': level, 
                     '# of students': num_students, 
                     'A':A,
                     'A-':Aminus,
                     'B+':Bplus,
                     'B':B,
                     'B-':Bminus,
                     'C+':Cplus,
                     'C':C,
                     'C-':Cminus,
                     'D+':Dplus, 
                     'D':D,
                     'F':F,
                     'NC':NC,
                     'N':N,
                     'P':P,
                     'S':S,
                     'W':W}) 

#######################################################
#####            Write a simple row Row           #####
#######################################################
def write_grade_simple_line (row_name):
    write_grade_row(row_name, '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '')
    
#######################################################
## Total Function to Generate Data to Write to Row ##
#######################################################
def write_grade_total_row (row_name, data_frame, catagory_value):
    total = data_frame[(data_frame['AvgGrade'] != '')]['ResearchID'].nunique()
    A = data_frame[(data_frame['AvgGrade'] == 'A')]['ResearchID'].nunique()
    Aminus = data_frame[(data_frame['AvgGrade'] == 'A-')]['ResearchID'].nunique()
    Bplus = data_frame[(data_frame['AvgGrade'] == 'B+')]['ResearchID'].nunique()
    B = data_frame[(data_frame['AvgGrade'] == 'B')]['ResearchID'].nunique()
    Bminus = data_frame[(data_frame['AvgGrade'] == 'B-')]['ResearchID'].nunique()
    Cplus = data_frame[(data_frame['AvgGrade'] == 'C+')]['ResearchID'].nunique()
    C = data_frame[(data_frame['AvgGrade'] == 'C')]['ResearchID'].nunique()
    Cminus = data_frame[(data_frame['AvgGrade'] == 'C-')]['ResearchID'].nunique()
    Dplus = data_frame[(data_frame['AvgGrade'] == 'D+')]['ResearchID'].nunique()
    D = data_frame[(data_frame['AvgGrade'] == 'D')]['ResearchID'].nunique()
    F = data_frame[(data_frame['AvgGrade'] == 'F')]['ResearchID'].nunique()
    NC = data_frame[(data_frame['AvgGrade'] == 'NC')]['ResearchID'].nunique()
    N = data_frame[(data_frame['AvgGrade'] == 'N')]['ResearchID'].nunique()
    P = data_frame[(data_frame['AvgGrade'] == 'P')]['ResearchID'].nunique()
    S = data_frame[(data_frame['AvgGrade'] == 'S')]['ResearchID'].nunique()
    W = data_frame[(data_frame['AvgGrade'] == 'W')]['ResearchID'].nunique()

    write_grade_row(row_name, catagory_value, total, A, Aminus, Bplus, B, Bminus, Cplus, C, Cminus, Dplus, D, F, NC, N, P, S, W)
    
#######################################################
## Generic Function to Generate Data to Write to Row ##
#######################################################
def write_grade_catagory_row (row_name, data_frame, catagory, catagory_value):
    total = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] != '')]['ResearchID'].nunique()
    A = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'A')]['ResearchID'].nunique()
    Aminus = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'A-')]['ResearchID'].nunique()
    Bplus = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'B+')]['ResearchID'].nunique()
    B = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'B')]['ResearchID'].nunique()
    Bminus = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'B-')]['ResearchID'].nunique()
    Cplus = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'C+')]['ResearchID'].nunique()
    C = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'C')]['ResearchID'].nunique()
    Cminus = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'C-')]['ResearchID'].nunique()
    Dplus = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'D+')]['ResearchID'].nunique()
    D = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'D')]['ResearchID'].nunique()
    F = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'F')]['ResearchID'].nunique()
    NC = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'NC')]['ResearchID'].nunique()
    N = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'N')]['ResearchID'].nunique()
    P = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'P')]['ResearchID'].nunique()
    S = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'S')]['ResearchID'].nunique()
    W = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'W')]['ResearchID'].nunique()

    write_grade_row(row_name, catagory_value, total, A, Aminus, Bplus, B, Bminus, Cplus, C, Cminus, Dplus, D, F, NC, N, P, S, W)

    
####################################################
#####     Set up and Create the Grades Report  #####
####################################################
english_df['NumDaysPresent'] = english_df['NumDaysPresent'].apply(pd.to_numeric, errors='coerce')
math_df['NumDaysPresent'] = math_df['NumDaysPresent'].apply(pd.to_numeric, errors='coerce')
english_df.fillna('', inplace=True)

with open(source_path + output_grade_report, "w", newline='') as csvfile:
    fieldnames = ['Student_Group', 'Category', '# of students', 
                  'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'F', 'NC', 'N', 'P', 'S', 'W']
                  
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    ##### Write English
    write_grade_simple_line('Bridge English')
    write_grade_total_row('English SBA Level', english_df, 'All English Bridge')
    write_grade_catagory_row ('English SBA Level', english_df, 'PerformanceLevel', 'L1')
    write_grade_catagory_row ('English SBA Level', english_df, 'PerformanceLevel', 'L2')
    write_grade_catagory_row ('English SBA Level', english_df, 'PerformanceLevel', 'L3')
    write_grade_catagory_row ('English SBA Level', english_df, 'PerformanceLevel', 'L4')
    write_grade_catagory_row ('English SBA Level', english_df, 'PerformanceLevel', 'BA')
    write_grade_catagory_row ('English SBA Level', english_df, 'PerformanceLevel', '')

    ##### Write English Gender
    write_grade_catagory_row ('English Gender', english_df, 'Gender', 'F')
    write_grade_catagory_row ('English Gender', english_df, 'Gender', 'M')

    ##### Write English Ethnic Race
    write_grade_catagory_row ('English Ethnic Race', english_df, 'FederalEthRaceRollupName', 'White')
    write_grade_catagory_row ('English Ethnic Race', english_df, 'FederalEthRaceRollupName', 'Hispanic/Latino of any race(s)')
    write_grade_catagory_row ('English Ethnic Race', english_df, 'FederalEthRaceRollupName', 'Two or More Races')
    write_grade_catagory_row ('English Ethnic Race', english_df, 'FederalEthRaceRollupName', 'Black/African American')
    write_grade_catagory_row ('English Ethnic Race', english_df, 'FederalEthRaceRollupName', 'Asian')
    write_grade_catagory_row ('English Ethnic Race', english_df, 'FederalEthRaceRollupName', 'Native Hawaiian/Other Pacific Islander')
    write_grade_catagory_row ('English Ethnic Race', english_df, 'FederalEthRaceRollupName', 'American Indian/Alaskan Native')

     ##### Write Math Data
    write_grade_simple_line('')
    write_grade_simple_line('')
    write_grade_simple_line('Bridge Math')
    write_grade_total_row('Math SBA Level', math_df, 'All Math Bridge')
    write_grade_catagory_row ('Math SBA Level', math_df, 'PerformanceLevel', 'L1')
    write_grade_catagory_row ('Math SBA Level', math_df, 'PerformanceLevel', 'L2')
    write_grade_catagory_row ('Math SBA Level', math_df, 'PerformanceLevel', 'L3')
    write_grade_catagory_row ('Math SBA Level', math_df, 'PerformanceLevel', 'L4')
    write_grade_catagory_row ('Math SBA Level', math_df, 'PerformanceLevel', 'BA')
    write_grade_catagory_row ('Math SBA Level', math_df, 'PerformanceLevel', '')

    ##### Write Math Gender
    write_grade_catagory_row ('Math Gender', math_df, 'Gender', 'F')
    write_grade_catagory_row ('Math Gender', math_df, 'Gender', 'M')

    ##### Write Math Ethnic Race
    write_grade_catagory_row ('Math Ethnic Race', math_df, 'FederalEthRaceRollupName', 'White')
    write_grade_catagory_row ('Math Ethnic Race', math_df, 'FederalEthRaceRollupName', 'Hispanic/Latino of any race(s)')
    write_grade_catagory_row ('Math Ethnic Race', math_df, 'FederalEthRaceRollupName', 'Two or More Races')
    write_grade_catagory_row ('Math Ethnic Race', math_df, 'FederalEthRaceRollupName', 'Black/African American')
    write_grade_catagory_row ('Math Ethnic Race', math_df, 'FederalEthRaceRollupName', 'Asian')
    write_grade_catagory_row ('Math Ethnic Race', math_df, 'FederalEthRaceRollupName', 'Native Hawaiian/Other Pacific Islander')
    write_grade_catagory_row ('Math Ethnic Race', math_df, 'FederalEthRaceRollupName', 'American Indian/Alaskan Native')

csvfile.close()
print('Execution Completed: ' + str(datetime.datetime.now().strftime('%m/%d/%Y,  %H:%M.%S')))


B or Better students went to a CTC?
Of the Bridge B or Better students, who took math/ English 1 st quarter, and then who got what grade- same for the comparison group. 



In [ ]:
#----------------------------------Work In Progress-------------------------------------------------------------#

###############################################
#####     Functions to write row data     #####
###############################################
def write_college_grade_row (row_name, category_value, num_students, A, percentA, B, percentB, 
                             C, percentC, D, percentD, F, percentF, Null, percentNull):
    writer.writerow({'Student_Group': row_name, 
                     'Category Value':category_value,
                     'Total Students': num_students,
                     'A':A,
                     '%A':percentA,
                     'B':B,
                     '%B':percentB,
                     'C':C,
                     '%C':percentC,
                     'D':D,
                     '%D':percentD,
                     'F':F,
                     '%F':percentF,
                     'Null':Null,
                     '%Null':percentNull})

def write_line (row_name):
    write_college_grade_row(row_name, '', '', '', '', '', '', '', '', '', '', '', '', '', '')
    
def write_reference_college_grade_row(row_name, category, category_value):
    temp_df[category + 'GradeNumber'] = temp_df[category + 'GradeNumber'].apply(pd.to_numeric, errors='coerce')
    
    num_students = temp_df.ResearchID.nunique()
    A = temp_df[temp_df[category + 'GradeNumber'] >= 3.66]['ResearchID'].nunique()
    percentA = str('%4.2f' %(100 * (A / num_students)))
    B = temp_df[(temp_df[category + 'GradeNumber'] < 3.66) & 
                (temp_df[category + 'GradeNumber'] >= 2.66) ]['ResearchID'].nunique()
    percentB =  str('%4.2f' %(100 * (B / num_students)))
    C = temp_df[(temp_df[category + 'GradeNumber'] < 2.66) & 
                (temp_df[category + 'GradeNumber'] >= 1.66) ]['ResearchID'].nunique()
    percentC = str('%4.2f' %(100 * (C / num_students)))
    D = temp_df[(temp_df[category + 'GradeNumber'] < 1.66) & 
                                    (temp_df[category + 'GradeNumber'] >= 1) ]['ResearchID'].nunique()
    percentD = str('%4.2f' %(100 * (D / num_students)))
    F = temp_df[temp_df[category + 'GradeNumber'] < 1]['ResearchID'].nunique()
    percentF = str('%4.2f' %(100 * (F / num_students)))
    Null = sum(pd.isnull(temp_df[category + 'GradeNumber']))
    percentNull = str('%4.2f' %(100 * (Null / num_students)))
    
    
    write_college_grade_row(row_name, category_value, num_students, A, percentA, B, percentB, 
                            C, percentC, D, percentD, F, percentF, Null, percentNull)
                            

####################################################
##### Set up and Create College Grades Report  #####
####################################################
with open(source_path + output_college_grade_report, "w", newline='') as csvfile:
    fieldnames = ['Student_Group', 'Category Value', 'Total Students', 'A', '%A', 'B', '%B', 
                  'C', '%C', 'D', '%D', 'F', '%F', 'Null', '%Null']
                  
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    write_line('English')
    
    ##### English Fall PreCollege
    temp_df = english_df[((english_df.StateCourseCode == 1069) | ((english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0))) & (english_df.FallPreCollege == 'Y')]
    write_reference_college_grade_row('English Fall PreCollege', 'FallPreCollege', 'All Students')

    temp_df = english_df[(english_df.StateCourseCode == 1069) & (english_df.FallPreCollege == 'Y')]
    write_reference_college_grade_row('English Fall PreCollege', 'FallPreCollege', 'Bridge Students')

    temp_df = english_df[(english_df.StateCourseCode == 1069) & (english_df.AvgGrade.isin(['A', 'A-', 'B+', 'B'])) & 
                         (english_df.FallPreCollege == 'Y')]
    write_reference_college_grade_row('English Fall PreCollege', 'FallPreCollege', 'Bridge B or Better Students')

    temp_df = english_df[(english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0) & (english_df.FallPreCollege == 'Y')]
    write_reference_college_grade_row('English Fall PreCollege', 'FallPreCollege', 'Non-Bridge Students')
    
    ##### English Winter PreCollege
    temp_df = english_df[((english_df.StateCourseCode == 1069) | ((english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0))) & (english_df.WinterPreCollege == 'Y')]
    write_reference_college_grade_row('English Winter PreCollege', 'WinterPreCollege', 'All Students')

    temp_df = english_df[(english_df.StateCourseCode == 1069) & (english_df.WinterPreCollege == 'Y')]
    write_reference_college_grade_row('English Winter PreCollege', 'WinterPreCollege', 'Bridge Students')

    temp_df = english_df[(english_df.StateCourseCode == 1069) & (english_df.AvgGrade.isin(['A', 'A-', 'B+', 'B'])) & 
                         (english_df.WinterPreCollege == 'Y')]
    write_reference_college_grade_row('English Winter PreCollege', 'WinterPreCollege', 'Bridge B or Better Students')

    temp_df = english_df[(english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0) & (english_df.WinterPreCollege == 'Y')]
    write_reference_college_grade_row('English Winter PreCollege', 'WinterPreCollege', 'Non-Bridge Students')
    
    ##### English Spring PreCollege
    temp_df = english_df[((english_df.StateCourseCode == 1069) | ((english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0))) & (english_df.SpringPreCollege == 'Y')]
    write_reference_college_grade_row('English Spring PreCollege', 'SpringPreCollege', 'All Students')

    temp_df = english_df[(english_df.StateCourseCode == 1069) & (english_df.SpringPreCollege == 'Y')]
    write_reference_college_grade_row('English Spring PreCollege', 'SpringPreCollege', 'Bridge Students')

    temp_df = english_df[(english_df.StateCourseCode == 1069) & (english_df.AvgGrade.isin(['A', 'A-', 'B+', 'B'])) & 
                         (english_df.SpringPreCollege == 'Y')]
    write_reference_college_grade_row('English Spring PreCollege', 'SpringPreCollege', 'Bridge B or Better Students')

    temp_df = english_df[(english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0) & (english_df.SpringPreCollege == 'Y')]
    write_reference_college_grade_row('English Spring PreCollege', 'SpringPreCollege', 'Non-Bridge Students')

    ##### English Fall College
    temp_df = english_df[((english_df.StateCourseCode == 1069) | ((english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0))) & (english_df.FallCollege == 'Y')]
    write_reference_college_grade_row('English Fall College', 'FallCollege', 'All Students')

    temp_df = english_df[(english_df.StateCourseCode == 1069) & (english_df.FallCollege == 'Y')]
    write_reference_college_grade_row('English Fall College', 'FallCollege', 'Bridge Students')

    temp_df = english_df[(english_df.StateCourseCode == 1069) & (english_df.AvgGrade.isin(['A', 'A-', 'B+', 'B'])) & 
                         (english_df.FallCollege == 'Y')]
    write_reference_college_grade_row('English Fall College', 'FallCollege', 'Bridge B or Better Students')

    temp_df = english_df[(english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0) & (english_df.FallCollege == 'Y')]
    write_reference_college_grade_row('English Fall College', 'FallCollege', 'Non-Bridge Students')

    ##### English Winter College
    temp_df = english_df[((english_df.StateCourseCode == 1069) | ((english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0))) & (english_df.WinterCollege == 'Y')]
    write_reference_college_grade_row('English Winter College', 'WinterCollege', 'All Students')

    temp_df = english_df[(english_df.StateCourseCode == 1069) & (english_df.WinterCollege == 'Y')]
    write_reference_college_grade_row('English Winter College', 'WinterCollege', 'Bridge Students')

    temp_df = english_df[(english_df.StateCourseCode == 1069) & (english_df.AvgGrade.isin(['A', 'A-', 'B+', 'B'])) & 
                         (english_df.WinterCollege == 'Y')]
    write_reference_college_grade_row('English Winter College', 'WinterCollege', 'Bridge B or Better Students')

    temp_df = english_df[(english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0) & (english_df.WinterCollege == 'Y')]
    write_reference_college_grade_row('English Winter College', 'WinterCollege', 'Non-Bridge Students')

    ##### English Spring College
    temp_df = english_df[((english_df.StateCourseCode == 1069) | ((english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0))) & (english_df.SpringCollege == 'Y')]
    write_reference_college_grade_row('English Spring College', 'SpringCollege', 'All Students')

    temp_df = english_df[(english_df.StateCourseCode == 1069) & (english_df.SpringCollege == 'Y')]
    write_reference_college_grade_row('English Spring College', 'SpringCollege', 'Bridge Students')

    temp_df = english_df[(english_df.StateCourseCode == 1069) & (english_df.AvgGrade.isin(['A', 'A-', 'B+', 'B'])) & 
                         (english_df.SpringCollege == 'Y')]
    write_reference_college_grade_row('English Spring College', 'SpringCollege', 'Bridge B or Better Students')

    temp_df = english_df[(english_df.StateCourseCode != 1069) & 
                         (english_df.SchoolYear == 2016) & (english_df.ExpectedGradYear == 2016) & 
                         (english_df.NumDaysPresent >= 90.0) & (english_df.SpringCollege == 'Y')]
    write_reference_college_grade_row('English Spring College', 'SpringCollege', 'Non-Bridge Students')



    
    write_line('')
    write_line('')
    write_line('')
    write_line('Math')
    
    ##### Math Fall PreCollege
    temp_df = math_df[((math_df.StateCourseCode == 2099) | ((math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0))) & (math_df.FallPreCollege == 'Y')]
    write_reference_college_grade_row('Math Fall PreCollege', 'FallPreCollege', 'All Students')

    temp_df = math_df[(math_df.StateCourseCode == 2099) & (math_df.FallPreCollege == 'Y')]
    write_reference_college_grade_row('Math Fall PreCollege', 'FallPreCollege', 'Bridge Students')

    temp_df = math_df[(math_df.StateCourseCode == 2099) & (math_df.AvgGrade.isin(['A', 'A-', 'B+', 'B'])) & 
                         (math_df.FallPreCollege == 'Y')]
    write_reference_college_grade_row('Math Fall PreCollege', 'FallPreCollege', 'Bridge B or Better Students')

    temp_df = math_df[(math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0) & (math_df.FallPreCollege == 'Y')]
    write_reference_college_grade_row('Math Fall PreCollege', 'FallPreCollege', 'Non-Bridge Students')
    
    ##### Math Winter PreCollege
    temp_df = math_df[((math_df.StateCourseCode == 2099) | ((math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0))) & (math_df.WinterPreCollege == 'Y')]
    write_reference_college_grade_row('Math Winter PreCollege', 'WinterPreCollege', 'All Students')

    temp_df = math_df[(math_df.StateCourseCode == 2099) & (math_df.WinterPreCollege == 'Y')]
    write_reference_college_grade_row('Math Winter PreCollege', 'WinterPreCollege', 'Bridge Students')

    temp_df = math_df[(math_df.StateCourseCode == 2099) & (math_df.AvgGrade.isin(['A', 'A-', 'B+', 'B'])) & 
                         (math_df.WinterPreCollege == 'Y')]
    write_reference_college_grade_row('Math Winter PreCollege', 'WinterPreCollege', 'Bridge B or Better Students')

    temp_df = math_df[(math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0) & (math_df.WinterPreCollege == 'Y')]
    write_reference_college_grade_row('Math Winter PreCollege', 'WinterPreCollege', 'Non-Bridge Students')
    
    ##### Math Spring PreCollege
    temp_df = math_df[((math_df.StateCourseCode == 2099) | ((math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0))) & (math_df.SpringPreCollege == 'Y')]
    write_reference_college_grade_row('Math Spring PreCollege', 'SpringPreCollege', 'All Students')

    temp_df = math_df[(math_df.StateCourseCode == 2099) & (math_df.SpringPreCollege == 'Y')]
    write_reference_college_grade_row('Math Spring PreCollege', 'SpringPreCollege', 'Bridge Students')

    temp_df = math_df[(math_df.StateCourseCode == 2099) & (math_df.AvgGrade.isin(['A', 'A-', 'B+', 'B'])) & 
                         (math_df.SpringPreCollege == 'Y')]
    write_reference_college_grade_row('Math Spring PreCollege', 'SpringPreCollege', 'Bridge B or Better Students')

    temp_df = math_df[(math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0) & (math_df.SpringPreCollege == 'Y')]
    write_reference_college_grade_row('Math Spring PreCollege', 'SpringPreCollege', 'Non-Bridge Students')

    ##### Math Fall College
    temp_df = math_df[((math_df.StateCourseCode == 2099) | ((math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0))) & (math_df.FallCollege == 'Y')]
    write_reference_college_grade_row('Math Fall College', 'FallCollege', 'All Students')

    temp_df = math_df[(math_df.StateCourseCode == 2099) & (math_df.FallCollege == 'Y')]
    write_reference_college_grade_row('Math Fall College', 'FallCollege', 'Bridge Students')

    temp_df = math_df[(math_df.StateCourseCode == 2099) & (math_df.AvgGrade.isin(['A', 'A-', 'B+', 'B'])) & 
                         (math_df.FallCollege == 'Y')]
    write_reference_college_grade_row('Math Fall College', 'FallCollege', 'Bridge B or Better Students')

    temp_df = math_df[(math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0) & (math_df.FallCollege == 'Y')]
    write_reference_college_grade_row('Math Fall College', 'FallCollege', 'Non-Bridge Students')

    ##### Math Winter College
    temp_df = math_df[((math_df.StateCourseCode == 2099) | ((math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0))) & (math_df.WinterCollege == 'Y')]
    write_reference_college_grade_row('Math Winter College', 'WinterCollege', 'All Students')

    temp_df = math_df[(math_df.StateCourseCode == 2099) & (math_df.WinterCollege == 'Y')]
    write_reference_college_grade_row('Math Winter College', 'WinterCollege', 'Bridge Students')

    temp_df = math_df[(math_df.StateCourseCode == 2099) & (math_df.AvgGrade.isin(['A', 'A-', 'B+', 'B'])) & 
                         (math_df.WinterCollege == 'Y')]
    write_reference_college_grade_row('Math Winter College', 'WinterCollege', 'Bridge B or Better Students')

    temp_df = math_df[(math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0) & (math_df.WinterCollege == 'Y')]
    write_reference_college_grade_row('Math Winter College', 'WinterCollege', 'Non-Bridge Students')

    ##### Math Spring College
    temp_df = math_df[((math_df.StateCourseCode == 2099) | ((math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0))) & (math_df.SpringCollege == 'Y')]
    write_reference_college_grade_row('Math Spring College', 'SpringCollege', 'All Students')

    temp_df = math_df[(math_df.StateCourseCode == 2099) & (math_df.SpringCollege == 'Y')]
    write_reference_college_grade_row('Math Spring College', 'SpringCollege', 'Bridge Students')

    temp_df = math_df[(math_df.StateCourseCode == 2099) & (math_df.AvgGrade.isin(['A', 'A-', 'B+', 'B'])) & 
                         (math_df.SpringCollege == 'Y')]
    write_reference_college_grade_row('Math Spring College', 'SpringCollege', 'Bridge B or Better Students')

    temp_df = math_df[(math_df.StateCourseCode != 2099) & 
                         (math_df.SchoolYear == 2016) & (math_df.ExpectedGradYear == 2016) & 
                         (math_df.NumDaysPresent >= 90.0) & (math_df.SpringCollege == 'Y')]
    write_reference_college_grade_row('Math Spring College', 'SpringCollege', 'Non-Bridge Students')

csvfile.close()
print('Execution Completed: ' + str(datetime.datetime.now().strftime('%m/%d/%Y,  %H:%M.%S')))
    